In [5]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 6 GPU(s) available.
We will use the GPU: NVIDIA A100 80GB PCIe


In [6]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install wget

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="google-bert/bert-base-uncased")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

In [8]:
import pandas as pd
"""
# Load the dataset into a pandas dataframe.
df = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(10)"""

'\n# Load the dataset into a pandas dataframe.\ndf = pd.read_csv("./cola_public/raw/in_domain_train.tsv", delimiter=\'\t\', header=None, names=[\'sentence_source\', \'label\', \'label_notes\', \'sentence\'])\n\n# Report the number of sentences.\nprint(\'Number of training sentences: {:,}\n\'.format(df.shape[0]))\n\n# Display 10 random rows from the data.\ndf.sample(10)'

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader
from datasets import load_dataset
import torch

# 데이터셋 로드 및 분할 
#dataset = load_dataset("수정해야함")
#train_dataset = dataset["train"]
#valid_dataset = dataset["validation"]

# 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 데이터 토크나이징 함수
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# 데이터셋 토크나이징
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_valid = valid_dataset.map(tokenize_function, batched=True)

# 데이터 로더 설정
train_loader = DataLoader(tokenized_train, batch_size=8)
valid_loader = DataLoader(tokenized_valid, batch_size=8)

# 모델 초기화
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 훈련 루프
model.train()
for epoch in range(4):  # 예시로 4 에포크 설정
    for batch in train_loader:
        inputs = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 from transformers import BertTokenizer, BertForSequenceClassification, AdamW                │
│    2 from torch.utils.data import DataLoader                                                     │
│ ❱  3 from datasets import load_dataset                                                           │
│    4 import torch                                                                                │
│    5                                                                                             │
│    6 # 데이터셋 로드 및 분할                                                                     │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'datasets'

In [ ]:
import json
from collections import Counter

# Load the original dataset with possible answers (this needs to be replaced with your actual data)
# Assuming `original_data.json` maps question IDs to a list of answers
# 파일 및 디렉토리 경로 설정
data_root = "/data2/KJE/okvqa"
image_data_path = '/data2/KJE/okvqa/okvqa_images(coco)/val2014'
repo_root = '/data2/KJE'
os.system(f'mkdir -p {repo_root}/OK-VQA/statement')

# 파일 이름 설정
fname_pattern = "OpenEnded_mscoco_val2014_questions.json"
file_path = f"{data_root}/okvqa_inputQuestion/{fname_pattern}"

with open('/data2/KJE/okvqa/okvqa_annotations/mscoco_val2014_annotations.json', 'r') as f:
    original_data = json.load(f)

def get_answer_data(answers_list):
    counter = Counter([ans['answer'].lower() for ans in answers_list])
    most_common_answer = counter.most_common(1)[0][0]
    distinct_answers = list(counter)
    return most_common_answer, distinct_answers


# Create the answer map
answer_map = {}
for annotation in annotations_data:
    most_common_answer, distinct_answers = get_answer_data(annotation['answers'])
    answer_map[annotation['question_id']] = {
        "most_common_answer": most_common_answer,
        "distinct_answers": distinct_answers
    }

# Process the questions
questions_file = f"{data_root}/okvqa_inputQuestion/OpenEnded_mscoco_val2014_questions.json"
with open(questions_file, 'r') as f:
    questions_data = json.load(f)['questions']

examples = []

for item in tqdm(questions_data):
    _id = item["question_id"]
    question_stem = item["question"]
    imageId = item["image_id"]

    # Retrieve the answer data from the map
    answer_data = answer_map.get(_id, None)
    if answer_data:
        most_common_answer = answer_data["most_common_answer"]
        distinct_answers = answer_data["distinct_answers"]
    else:
        most_common_answer = 'N/A'
        distinct_answers = ['N/A']


# Load the preprocessed data
with open('/data2/KJE/OK-VQA/statement/OpenEnded_mscoco_val2014_questions_processed.json', 'r') as f:
    preprocessed_data = json.load(f)

# Update the statement field in the preprocessed data
for entry in preprocessed_data:
    question_id = entry['id']
    if entry['statement'] == "N/A":
        # Update the statement with the most common answer, or keep "N/A" if no answer is found
        entry['statement'] = answer_map.get(question_id, "N/A")

# Save the updated data to a new JSON file
with open('/data2/KJE/OK-VQA/statement/updated_OpenEnded_mscoco_val2014_questions_processed.json', 'w') as f:
    json.dump(preprocessed_data, f, indent=4, ensure_ascii=False)

print("Updated JSON file has been saved.")


###################################################################
# Define the data directories
data_root = "/path/to/okvqa"  # Update this path to the root directory of the OK-VQA dataset
repo_root = '/path/to/output'  # Update this path to your desired output directory
os.makedirs(f'{repo_root}/OK-VQA/statement', exist_ok=True)

# Read the annotations and create a mapping from question_id to answers
annotations_file = f"{data_root}/okvqa_annotations/mscoco_val2014_annotations.json"
with open(annotations_file, 'r') as f:
    annotations_data = json.load(f)["annotations"]

# Helper function to get the most common answer and all distinct answers
def get_answer_data(answers_list):
    counter = Counter([ans['answer'].lower() for ans in answers_list])
    most_common_answer = counter.most_common(1)[0][0]
    distinct_answers = list(counter)
    return most_common_answer, distinct_answers

# Create the answer map
answer_map = {}
for annotation in annotations_data:
    most_common_answer, distinct_answers = get_answer_data(annotation['answers'])
    answer_map[annotation['question_id']] = {
        "most_common_answer": most_common_answer,
        "distinct_answers": distinct_answers
    }

# Process the questions
questions_file = f"{data_root}/okvqa_inputQuestion/OpenEnded_mscoco_val2014_questions.json"
with open(questions_file, 'r') as f:
    questions_data = json.load(f)['questions']

examples = []

for item in tqdm(questions_data):
    _id = item["question_id"]
    question_stem = item["question"]
    imageId = item["image_id"]

    # Retrieve the answer data from the map
    answer_data = answer_map.get(_id, None)
    if answer_data:
        most_common_answer = answer_data["most_common_answer"]
        distinct_answers = answer_data["distinct_answers"]
    else:
        most_common_answer = 'N/A'
        distinct_answers = ['N/A']

    # Construct the example object
    ex_obj = {
        "id": str(_id),
        "question": {
            "stem": question_stem,
            "choices": [{"label": chr(65 + i), "text": ans} for i, ans in enumerate(distinct_answers)]
        },
        "answerKey": most_common_answer,
        "image": f"{imageId}.jpg"
    }
    examples.append(ex_obj)

In [4]:
import json
from collections import Counter

# Load the original dataset with possible answers (this needs to be replaced with your actual data)
# Assuming `original_data.json` maps question IDs to a list of answers
with open('/data2/KJE/okvqa/okvqa_annotations/mscoco_val2014_annotations.json', 'r') as f:
    original_data = json.load(f)

# Create a mapping of question IDs to their most common answer
answer_mapping = {}
for entry in original_data:
    try:
        question_id = entry['question_id']
    except:
        pass
        
    answers = [ans['answer'] for ans in entry.get('answers', [])]
    most_common_answer = Counter(answers).most_common(1)[0][0] if answers else "N/A"
    answer_mapping[question_id] = most_common_answer

# Load the preprocessed data
with open('/data2/KJE/OK-VQA/statement/OpenEnded_mscoco_val2014_questions_processed.json', 'r') as f:
    preprocessed_data = json.load(f)

# Update the statement field in the preprocessed data
for entry in preprocessed_data:
    question_id = entry['id']
    if entry['statement'] == "N/A":
        # Update the statement with the most common answer, or keep "N/A" if no answer is found
        entry['statement'] = answer_mapping.get(question_id, "N/A")

# Save the updated data to a new JSON file
with open('updated_OpenEnded_mscoco_val2014_questions_processed.json', 'w') as f:
    json.dump(preprocessed_data, f, indent=4, ensure_ascii=False)

print("Updated JSON file has been saved.")


AttributeError: 'str' object has no attribute 'get'

In [9]:
import json
from collections import Counter

# Load the original dataset with annotations
with open('/data2/KJE/okvqa/okvqa_annotations/mscoco_val2014_annotations.json', 'r') as f:
    original_data = json.load(f)
    annotations = original_data['annotations']

# Create a mapping of question IDs to their most common answer
answer_mapping = {}
for ann in annotations:
    question_id = ann['question_id']
    # Count the frequency of each answer
    answer_counter = Counter([a['answer'] for a in ann['answers']])
    # Find the most common answer
    most_common_answer = answer_counter.most_common(1)[0][0]
    answer_mapping[str(question_id)] = most_common_answer
    #print(answer_mapping)

# Load the preprocessed data
with open('/data2/KJE/OK-VQA/statement/OpenEnded_mscoco_val2014_questions_processed.json', 'r') as g:
    preprocessed_data = json.load(g)

# Update the statement field in the preprocessed data
for item in preprocessed_data:
    question_id = str(item['id'])
    print(question_id)
    # Update the statement with the most common answer
    if question_id in answer_mapping:
        item['statement'] = answer_mapping[question_id]
    else:
        item['statement'] = "N/A"

# Save the updated data to a new JSON file
with open('updated_OpenEnded_mscoco_val2014_questions_processed.json', 'w') as h:
    json.dump(preprocessed_data, h, indent=4)


print("Updated JSON file has been saved.")


2971475
3397615
3575865
949225
2076115
5723996
5759705
3045575
2183655
2863135
299845
115115
3234605
5169165
217115
3133865
4026395
4575845
4883775
3017535
1833195
3970635
4976605
2357845
3190735
5144685
4371265
5182875
3804595
5002005
3156685
3736625
4293695
4273415
3938365
179845
3268375
2780325
1110325
5096415
3413095
1121605
2645405
1089825
2929715
1902045
3774865
1282805
1318415
1305165
2845485
5069455
4247445
5681485
857725
1396845
4732085
3030245
5342525
1649245
1197855
511915
1943135
3325125
3763585
3959785
2162545
2913705
3304055
505145
5666345
1569995
4535205
3282845
2335535
3589765
1050405
5379826
2352215
4854245
2864225
5192715
1449855
5605665
3430685
1527025
1603935
2597615
3883985
4071465
5525735
1000065
5685555
2873285
5649115
4471175
3294555
1365725
5775245
2707535
2231575
3087645
2940305
2741565
5522215
4684875
5156125
2085245
1970975
2365425
1041195
2264595
2749575
1341195
1708525
1667045
4100045
639735
979885
4959965
2980515
2921885
5735275
5461715
3989925
3159865
41